# Experiment: Try to find class specific neurons in the neural network

In [1]:
import tensorflow as tf
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-08-18 15:41:56.662888: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 15:41:56.703475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 15:41:57.460521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-08-18 15:41:58.014879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 15:41:58.045018: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 15:41:58.045344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# The data

In [2]:
image_size = (180, 180)
batch_size = 16 # My GPU could only handle this BS

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Using 4682 files for validation.


2023-08-18 15:41:58.485767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 15:41:58.486223: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-18 15:41:58.486298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# Model stuff

In [3]:
model = keras.models.load_model(f"./models_from_server/model_cats_dogs/model/save_at_100.keras")

model.compile(
    # optimizer=keras.optimizers.Adam(learning_rate),
    # loss="binary_crossentropy", 
    metrics=["accuracy"],
)

In [4]:
model.evaluate(val_ds)

2023-08-18 15:41:59.418095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


  1/293 [..............................] - ETA: 4:14 - loss: 0.0000e+00 - accuracy: 1.0000

2023-08-18 15:41:59.788014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


 19/293 [>.............................] - ETA: 12s - loss: 0.0000e+00 - accuracy: 0.9704

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


...

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


225/293 [======================>.......] - ETA: 3s - loss: 0.0000e+00 - accuracy: 0.9589

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


293/293 [==============================] - 14s 46ms/step - loss: 0.0000e+00 - accuracy: 0.9579


[0.0, 0.957923948764801]

# New stuff

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 180, 180, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 180, 180, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 90, 90, 128)          3584      ['rescaling[0][0]']           
                                                                                                  
 batch_normalization (Batch  (None, 90, 90, 128)          512       ['conv2d[0][0]']              
 Normalization)                                                                               

In [6]:
len(val_ds)

293

In [8]:
# Validation loop

import time
from tqdm import tqdm

BC_loss = keras.metrics.BinaryCrossentropy(from_logits=True)
binary_accuracy = keras.metrics.BinaryAccuracy()

for i, (x_batch_val, y_batch_val) in tqdm(enumerate(val_ds), total=len(val_ds)):

# for x_batch_val, y_batch_val in val_ds:
    val_logits = model(x_batch_val, training=False)
    #

    BC_loss.update_state(y_batch_val, val_logits)
    binary_accuracy.update_state(y_batch_val, val_logits)
    
    # Update val metrics
    # try:
    #     # BC_loss.update_state(tf.reshape(val_logits, (16,1)), val_logits)
    #     # binary_accuracy.update_state(tf.reshape(val_logits, (16,1)), val_logits)

    #     BC_loss.update_state(y_batch_val, val_logits)
    #     binary_accuracy.update_state(y_batch_val, val_logits)

    # except:
    #     # BC_loss.update_state(tf.reshape(val_logits, (len(val_logits),1)), val_logits)
    #     # binary_accuracy.update_state(tf.reshape(val_logits, (len(val_logits),1)), val_logits)

    #     BC_loss.update_state(y_batch_val, val_logits)
    #     binary_accuracy.update_state(y_batch_val, val_logits)
 
    # print(val_acc_metric.result())

    # if i == 10:
        
    #     print(val_logits)
    #     print(val_logits.shape)
    #     print("======================")
    #     print(y_batch_val)
    #     print("======================")
    #     print(tf.reshape(val_logits, (16,1)))
    #     print("======================")

    #     print(tf.reshape(y_batch_val, [-1]))
    #     # print(tf.reshape(val_logits, [-1]).shape)

    #     print(len(val_logits))

    #     break
    
# BC_loss = val_acc_metric.result()
# val_acc_metric.reset_states()
print(f"Binary crossentropy loss: {BC_loss.result()}")
print(f"Binary accuracy: {binary_accuracy.result()}")

  0%|          | 0/293 [00:00<?, ?it/s]

100%|██████████| 293/293 [00:14<00:00, 19.60it/s]

Binary crossentropy loss: 0.5243650078773499
Binary accuracy: 0.957923948764801
